### ***Logic***

##### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
##### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
     * Không quá 72h.
##### Backlog lấy
##### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
     * HCM -> HN: 12h.
     * Các tỉnh còn lại: 32h.
##### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
     * Giao hàng tại HCM HN: 36h.
     * Giao hàng tại các tỉnh còn lại: 56h.

In [40]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiKien',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[['TGDongKien', 'TGCapNhat', 'TGNhanKien', 'TGKetThuc']].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
# join with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien']], on='MaDH', how='left')
# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery = data[data['TrangThai'].isin([
    'Đang giao hàng', 'Giao không thành công',
    'Chờ xác nhận giao lại'
]) | ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] == data['KhoHienTai'])) |\
                          ((data['TrangThai'] == 'Tạo thành công') & ((data['ThoiGianKetThucLay'] != '<nil>') ^\
                          (data['ThoiGianKetThucGiao'] != '<nil>')))]
delivery['LoaiBacklog'] = 'Xử lý giao'
delivery['N0'] = delivery[delivery['KhoLay'] == delivery['KhoHienTai']]['ThoiGianKetThucLay']
delivery['N0'] = delivery[~(delivery['KhoLay'] == delivery['KhoHienTai'])]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(days=(120/24))
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(days=9999))


# return backlog (backlog trả)
returned = data[data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn không thành công'
    ]) | ((data['TrangThai'] == 'Trả hàng') & (((data['KhoTra'] != '<nil>') & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'] == '<nil>') & (data['KhoLay'] == data['KhoHienTai'])))) |\
          ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucGiao'] != '<nil>'))]
returned['LoaiBacklog'] = 'Xử lý trả'
returned['N0'] = returned[returned['KhoHienTai'] == returned['KhoGiao']]['ThoiGianKetThucGiao']
returned['N0'] = returned[~(returned['KhoHienTai'] == returned['KhoGiao'])]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(days=(72/24))
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(days=9999))


# pick_backlog (backlog lấy)
pickup = data[data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])]
pickup['LoaiBacklog'] = 'Xử lý lấy'

pickup['N0'] = pickup[pickup['MaKH'].isin(ecoms)]['ThoiGianTao']
pickup['N0'] = pickup[~pickup['MaKH'].isin(ecoms)]['ThoiGianTaoChuyenDoi']
pickup['N+'] = pickup['N0'] + pd.Timedelta(days=3)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(days=9999))

# transit_backlog (backlog luân chuyển)
transit = data[(data['TrangThai'] == 'GHN đã lấy') |\
                         ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] != data['KhoHienTai']))]

transit['LoaiBacklog'] = 'Xử lý luân chuyển'
transit['N0'] = transit['ThoiGianKetThucLay']
transit['N+'] = transit[(transit['DenTinh'] == 'Hà Nội') | (transit['DenTinh'] == 'Hồ Chí Minh')]['N0'] + pd.Timedelta(days=12/24)
transit['N+'] = transit[~((transit['DenTinh'] == 'Hà Nội') | (transit['DenTinh']== 'Hồ Chí Minh'))]['N0'] + pd.Timedelta(days=32/24)
transit['Aging'] = (datetime.today() - transit['N+']).fillna(pd.Timedelta(days=9999))


# returned_transit_backlog (backlog luân chuyển trả hàng)
returned_transit = export[export['TrangThai'].isin([
    'Trả hàng', 'Chờ trả hàng'
]) & (((data['KhoTra'] == '<nil>') & (data['KhoLay'] != data['KhoHienTai'])) |\
      ((data['KhoTra'] != '<nil>') & (data['KhoTra'] != data['KhoHienTai'])))]
returned_transit['LoaiBacklog'] = 'Xử lý luân chuyển trả'
returned_transit['N0'] = returned_transit['ThoiGianKetThucGiao']
returned_transit['N+'] = returned_transit[
    (returned_transit['DenTinh'] == 'Hà Nội') | (returned_transit['DenTinh'] == 'Hồ Chí Minh')]['N0'] + pd.Timedelta(days=36/24)
returned_transit['N+'] = returned_transit[
    ~((returned_transit['DenTinh'] == 'Hà Nội') | (returned_transit['DenTinh'] == 'Hồ Chi Minh'))
]['N0'] + pd.Timedelta(days=56/24)
returned_transit['Aging'] = (datetime.today() - returned_transit['N+']).fillna(pd.Timedelta(days=9999))


# composite data
data = delivery.append([returned, pickup, transit, returned_transit])
data['e-commerces'] = np.nan

# name of e-commerces exchange
data.loc[data['MaKH'].isin(shopee_codes), 'e-commerces'] = 'Shopee'
# shopee_name['e-commerces'] = 'Shopee'
data.loc[data['MaKH'].isin(sendo_codes), 'e-commerces'] = 'Sendo'
# sendo_name['e-commerces'] = 'Sendo'
data.loc[data['MaKH'].isin(tiki_codes), 'e-commerces'] = 'Tiki'
# tiki_name['e-commerces'] = 'Tiki'
data.loc[data['MaKH'].isin(lazada_codes), 'e-commerces'] = 'Lazada'

# lazada_name['e-commerces'] = 'Lazada'
# data['e-commerces'] = shopee_name.append([sendo_name, tiki_name, lazada_name])

# data['ToanTrinh'] = data[data['MaKH'].isin(floors)]['N0'] - datetime.today()
data[data['e-commerces'] == 'Shopee']


,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,LoaiBacklog,N0,N+,Aging,e-commerces
52,8FHFQLL4U,2407622972850176676,2020-08-02 22:52:48,2020-08-03 08:00:00,2020-08-03 17:59:09,2020-08-04 06:37:41,NaT,2020-08-05 23:00:00,NaT,NaN,Quận Bình Tân,Hồ Chí Minh,E,Quận 4,Hồ Chí Minh,E,Bưu Cục 291 Nguyễn Sơn-Q.Tân Phú-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 327E Lê Văn Quới-Q.Bình Tân-HCM,Giao không thành công,900,10x60x15,18692,'04/08/2020 06:38:07 - GHN-DFC1A4|KHÔNG LIÊN L...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,1 Ngày,1,1,0,NaN,2039440,NaN,'noithatinox',1154025,20KDE05496,2329.0,2275.0,2020-08-04 00:49:25,Xử lý giao,2020-08-04 00:49:25,2020-08-09 00:49:25,10 days 16:31:29.153232,Shopee
82,8FHA6YD7Y,2412466328375296517,2020-08-09 15:15:45,2020-08-10 08:00:00,2020-08-09 21:28:58,NaT,NaT,2020-08-13 23:00:00,NaT,NaN,Huyện Củ Chi,Hồ Chí Minh,E,Thành phố Nha Trang,Khánh Hòa,D,Bưu Cục 242 Tỉnh Lộ 2-Củ Chi-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Kho Khách hàng lớn Hồ Chí Minh 02,Lưu kho,6748,19x111x16,18692,NaN,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,1,0,0,NaN,1868573,NaN,'Kho Strading Củ Chi',1154025,20GGV48349,2279.0,2275.0,NaT,Xử lý giao,NaT,NaT,9999 days 00:00:00,Shopee
83,8FH9967QY,2412505587056643264,2020-08-09 16:33:43,2020-08-10 08:00:00,2020-08-09 21:00:54,NaT,NaT,2020-08-12 23:00:00,NaT,NaN,Huyện Củ Chi,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 242 Tỉnh Lộ 2-Củ Chi-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Kho Khách hàng lớn Hồ Chí Minh 02,Lưu kho,69552,28x138x90,18692,NaN,'Không cho xem hàng. Khối lượng nhận hàng tối ...,1 Ngày,1,0,0,NaN,1868573,NaN,'Kho Strading Củ Chi',1154025,20RXF40734,1327.0,2275.0,2020-08-10 03:27:54,Xử lý giao,2020-08-10 03:27:54,2020-08-15 03:27:54,4 days 13:53:00.153232,Shopee
84,8FHQQUXAS,2412603465338880088,2020-08-09 19:48:10,2020-08-13 08:00:00,2020-08-14 16:21:08,2020-08-17 07:56:03,NaT,2020-08-18 17:54:00,NaT,NaN,Thị Xã Phú Mỹ,Bà Rịa – Vũng Tàu,F,Quận 7,Hồ Chí Minh,E,Bưu Cục Phú Mỹ-Bà Rịa Vũng Tàu,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Phú Mỹ-Bà Rịa Vũng Tàu,Đang giao hàng,110,10x10x10,18692,'13/08/2020 09:09:36 - GHN-PFA2A1|KHÔNG LIÊN L...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,2,2,0,NaN,1879750,1880433,'phu24102017',1154025,20UXG58349,2329.0,2275.0,2020-08-16 02:59:42,Xử lý giao,2020-08-16 02:59:42,2020-08-21 02:59:42,-2 days +14:21:12.153232,Shopee
91,8FHF76SAH,2412514562850816740,2020-08-09 16:51:31,2020-08-10 08:00:00,2020-08-13 09:48:13,2020-08-15 14:36:17,NaT,2020-08-16 17:54:00,NaT,NaN,Quận Thanh Xuân,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 57 Hạ Đình-Q.Thanh Xuân-HN 03,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 57 Hạ Đình-Q.Thanh Xuân-HN 03,Đang giao hàng,200,10x10x10,18692,'10/08/2020 08:51:47 - GHN-PFA1A0|THAY ĐỔI THÔ...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,4,2,0,NaN,1871315,245583,'Baby Closet by T&T',1154025,20GRJ96555,2329.0,2275.0,2020-08-15 14:05:19,Xử lý giao,2020-08-15 14:05:19,2020-08-20 14:05:19,-1 days +03:15:35.153232,Shopee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,8FH3DDLAAU,2415442707181568825,2020-08-13 17:49:13,2020-08-14 08:00:00,2020-08-14 17:51:52,2020-08-15 08:01:17,2020-08-15 18:30:04,2020-08-18 10:42:00,NaT,'KHÁCH TỪ CHỐI NHẬN/Khách đổi ý',Quận Tân Bình,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 353A Nguyễn Thái Bình-Q.Tân Bình-HCM 01,

In [ ]:
# export data backlog to google sheets
# data export
data_sheets = data[[
    'MaDH',
    'MaKH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'TGNhanKien',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'e-commerces',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)

# update data to google sheets
wks.clear()
df2g.upload(
    data_sheets[data_sheets['Aging'] >= pd.Timedelta(days=0)],
    SPREADSHEET_ID, WKS_NAME,
    credentials=credentials,
    row_names=False,
)


def iter_df(df):
    """Generator for df."""
    for val in df.columns:
        yield val
    for row in df.to_numpy():
        for val in row:
            if pd.isna(val):
                yield ''
            else:
                yield val
                

# df_to_sheets(df, wks)


In [ ]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)